## Try running vanilla MD on ala dipeptide in solvent at the first thermostate and see if the problem happens

In [1]:
from tqdm import tqdm_notebook
import openmmtools

In [2]:
import pickle
import os
from openeye import oechem
from perses.annihilation.rest import RESTTopologyFactoryV3
from perses.annihilation.lambda_protocol import RESTStateV3
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.testsystems import AlanineDipeptideExplicit


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.5 jupyter extensions


In [43]:
length = 1
move_length = 1
timestep = 4
radius = 0.4


In [52]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
positions = ala.positions
topology = ala.topology
system.addForce(openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50))

# Create REST system
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
REST_system = factory.REST_system


INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 3.2893431387452243
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]


In [54]:
system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [55]:
system.getForce(3).getEwaldErrorTolerance()

1e-05

In [4]:
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 300.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV3.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

# Create thermodynamics states
sampler_state =  SamplerState(positions, box_vectors=system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in [temperatures[0], temperatures[-1]]:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state, max_iterations=0)
    sampler_state_list.append(copy.deepcopy(sampler_state))


In [5]:
REST_system.getForces()

[<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ad6e0ef5150> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad6e0ef53f0> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ad6e0ef5330> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ad6e0ef54b0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad6e0ef5540> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad6e0ef55a0> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad6e0ef5630> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ad6e0ef56c0> >]

In [6]:
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin,
                                              collision_rate=1 / unit.picoseconds,
                                              timestep= 4 * unit.femtoseconds)
context_cache = cache.global_context_cache
context, integrator = context_cache.get_context(thermodynamic_state_list[0], integrator)


In [7]:
sampler_state_list[0].apply_to_context(context)

In [8]:
old, new = list(), list()
for _ in tqdm_notebook(range(int(125000/250))):
    integrator.step(250)
    
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
#     old_pos = np.asarray(htf.old_positions(pos))
#     new_pos = np.asarray(htf.new_positions(pos))
#     old.append(old_pos)
#     new.append(new_pos)
    old.append(pos)

/scratch/lsftmp/3143052.tmpdir/ipykernel_4220/3652335998.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(125000/250))):


  0%|          | 0/500 [00:00<?, ?it/s]

In [9]:
with open("ala_300K.npy", 'wb') as f:
    np.save(f, old)
# with open("new_1200K.npy", 'wb') as f:
#     np.save(f, new)

In [10]:
import mdtraj as md

In [11]:
traj = md.Trajectory(np.array(old), md.Topology.from_openmm(ala.topology))
traj.save("ala_300K.dcd")
traj[0].save("ala_300K.pdb")

## 1200 K

In [12]:
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin,
                                              collision_rate=1 / unit.picoseconds,
                                              timestep= 4 * unit.femtoseconds)
context_cache = cache.global_context_cache
context, integrator = context_cache.get_context(thermodynamic_state_list[1], integrator)


In [13]:
sampler_state_list[1].apply_to_context(context)

In [14]:
old, new = list(), list()
for _ in tqdm_notebook(range(int(125000/250))):
    integrator.step(250)
    
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
#     old_pos = np.asarray(htf.old_positions(pos))
#     new_pos = np.asarray(htf.new_positions(pos))
#     old.append(old_pos)
#     new.append(new_pos)
    old.append(pos)

/scratch/lsftmp/3143052.tmpdir/ipykernel_4220/3652335998.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(125000/250))):


  0%|          | 0/500 [00:00<?, ?it/s]

In [15]:
with open("ala_1200K.npy", 'wb') as f:
    np.save(f, old)
# with open("new_1200K.npy", 'wb') as f:
#     np.save(f, new)

In [16]:
import mdtraj as md

In [17]:
traj = md.Trajectory(np.array(old), md.Topology.from_openmm(ala.topology))
traj.save("ala_1200K.dcd")
traj[0].save("ala_1200K.pdb")

## Try running vanilla MD on bnbs at the first thermostate and see if the problem happens

In [18]:
from tqdm import tqdm_notebook
import openmmtools

In [19]:
import pickle
import os
from openeye import oechem
from perses.annihilation.rest import RESTTopologyFactoryV3
from perses.annihilation.lambda_protocol import RESTStateV3
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
# cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openmmtools.testsystems import AlanineDipeptideExplicit


In [20]:
length = 1
move_length = 1
timestep = 4
radius = 0.4


In [24]:
# Load rhtf
htf = pickle.load(open("T42A_complex_0.pickle", "rb" ))
system = htf.hybrid_system
positions = htf.hybrid_positions


In [50]:
system.getForce(4).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [51]:
system.getForce(4).getEwaldErrorTolerance()

0.00025

In [25]:
# Build REST factory
for res in htf.hybrid_topology.residues:
    if res.resSeq == 83 and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
traj = md.Trajectory(np.array(htf.hybrid_positions), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
rest_atoms = [int(atom) for atom in rest_atoms]
factory = RESTTopologyFactoryV3(system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)

INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 2.7569734238004693
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]


In [26]:
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 300.0 * unit.kelvin  # Minimum temperature.
T_max = 1200 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTStateV3.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

# Create thermodynamics states
sampler_state =  SamplerState(positions, box_vectors=system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in [temperatures[0], temperatures[-1]]:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state, max_iterations=0)
    sampler_state_list.append(copy.deepcopy(sampler_state))


In [27]:
REST_system.getForces()

[<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ad713f308d0> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad713f308a0> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ad713f307b0> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ad713f304b0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad713f304e0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad713f30570> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad713f30870> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ad713f30900> >]

In [28]:
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin,
                                              collision_rate=1 / unit.picoseconds,
                                              timestep= 4 * unit.femtoseconds)
context_cache = cache.global_context_cache
context, integrator = context_cache.get_context(thermodynamic_state_list[0], integrator)


In [29]:
sampler_state_list[0].apply_to_context(context)

In [30]:
old, new = list(), list()
for _ in tqdm_notebook(range(int(12500/250))): ## NOTE THIS MAY HAVE BEEN TO SHORT TO SEE THE ISSUE
    integrator.step(250)
    
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    old_pos = np.asarray(htf.old_positions(pos))
#     new_pos = np.asarray(htf.new_positions(pos))
    old.append(old_pos)
#     new.append(new_pos)

/scratch/lsftmp/3143052.tmpdir/ipykernel_4220/379063010.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(12500/250))):


  0%|          | 0/50 [00:00<?, ?it/s]

In [31]:
with open("T42A_old_300K.npy", 'wb') as f:
    np.save(f, old)
# with open("new_1200K.npy", 'wb') as f:
#     np.save(f, new)

In [32]:
import mdtraj as md

In [33]:
traj = md.Trajectory(np.array(old), md.Topology.from_openmm(htf._topology_proposal.old_topology))
traj.save("T42A_old_300K.dcd")
traj[0].save("T42A_old_300K.pdb")

## 1200 K

In [39]:
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin,
                                              collision_rate=1 / unit.picoseconds,
                                              timestep= 4 * unit.femtoseconds)
context_cache = cache.global_context_cache
context, integrator = context_cache.get_context(thermodynamic_state_list[1], integrator)


In [40]:
sampler_state_list[1].apply_to_context(context)

In [41]:
old, new = list(), list()
for _ in tqdm_notebook(range(int(50000/250))):
    integrator.step(250)
    
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    old_pos = np.asarray(htf.old_positions(pos))
#     new_pos = np.asarray(htf.new_positions(pos))
    old.append(old_pos)
#     new.append(new_pos)

/scratch/lsftmp/3143052.tmpdir/ipykernel_4220/664631440.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for _ in tqdm_notebook(range(int(50000/250))):


  0%|          | 0/200 [00:00<?, ?it/s]

In [42]:
with open("T42A_old_1200K.npy", 'wb') as f:
    np.save(f, old)
# with open("new_1200K.npy", 'wb') as f:
#     np.save(f, new)

In [43]:
import mdtraj as md

In [44]:
traj = md.Trajectory(np.array(old), md.Topology.from_openmm(htf._topology_proposal.old_topology))
traj.save("T42A_old_1200K.dcd")
traj[0].save("T42A_old_1200K.pdb")

In [46]:
context.getSystem().getForces()

[<openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad72a086540> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2ad72a086600> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2ad72a0866c0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad72a086840> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2ad72a086900> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2ad7284c63f0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ad7284c6090> >,
 <openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2ad7284c6120> >]

In [58]:
REST_system.getForce(5).getUseSwitchingFunction()

True